In [65]:
import numpy as np
import os
import glob
import matplotlib.image as mpimg

### Tensor Functions (Fold/Unfold/Multi/HOSVD)

In [66]:
import numpy as np

# unfold function
def unfold(tensor, mod):

    if (mod == 1):
        unfolded_tensor = tensor[:, 0, :]
        for i in range(tensor.shape[1]-1):
            unfolded_tensor = np.concatenate((unfolded_tensor, tensor[:, i+1, :]), axis=1)

    elif (mod == 2):
        unfolded_tensor = tensor[:, :, 0].T
        for i in range(tensor.shape[2]-1):
            unfolded_tensor = np.concatenate((unfolded_tensor, tensor[:, :, i+1].T), axis=1)

    else:
        unfolded_tensor = tensor[0, :, :].T
        for i in range(tensor.shape[0]-1):
            unfolded_tensor = np.concatenate((unfolded_tensor, tensor[i+1, :, :].T), axis=1)

    return unfolded_tensor

In [67]:
# fold function
def fold(matrix, tensor_shape, mode):
    if mode == 1:
        return matrix.reshape(tensor_shape)
    elif mode == 2:
        return matrix.reshape(tensor_shape[1], tensor_shape[0], tensor_shape[2]).transpose(1, 0, 2)
    elif mode == 3:
        return matrix.reshape(tensor_shape[2], tensor_shape[0], tensor_shape[1]).transpose(1, 2, 0)

In [68]:
def multiplication(tensor, matrix, mod):
    if mod == 1:
        unfolded_matrix = unfold(tensor, 1)
        multiplied_matrix = np.dot(matrix, unfolded_matrix)
        tensor_resized = np.resize(tensor, (matrix.shape[0], tensor.shape[1], tensor.shape[2]))
        folded_matrix = fold(multiplied_matrix, tensor_resized.shape, 1)
    elif mod == 2:
        unfolded_matrix = unfold(tensor, 2)
        multiplied_matrix = np.dot(matrix, unfolded_matrix)
        tensor_resized = np.resize(tensor, (tensor.shape[0], matrix.shape[0], tensor.shape[2]))
        folded_matrix = fold(multiplied_matrix, tensor_resized.shape, 2)
    elif mod == 3:
        unfolded_matrix = unfold(tensor, 3)
        multiplied_matrix = np.dot(matrix, unfolded_matrix)
        tensor_resized = np.resize(tensor, (tensor.shape[0], tensor.shape[1], matrix.shape[0]))
        folded_matrix = fold(multiplied_matrix, tensor_resized.shape, 3)
    return folded_matrix

In [69]:
## HOSVD function
def HOSVD(A):

    U1, s1, v = np.linalg.svd(unfold(A,1), full_matrices=False)
    U2, s2, v = np.linalg.svd(unfold(A,2), full_matrices=False)
    U3, s3, v = np.linalg.svd(unfold(A,3), full_matrices=False)

    S = multiplication(multiplication(multiplication(A,U1.T,1), U2.T,2), U3.T,3)
    
    return S, U1, U2, U3

In [70]:
# Get unique of lists
def unique(list):
    unique_list = []
    for i in list:
        if i not in unique_list:
            unique_list.append(i)
    return unique_list


Read Train images and create Tensor A.

In [71]:
images_list=[]
for img in glob.glob("D:/Projects/train/*.gif"):
    images=mpimg.imread(img)
    images_list.append(images)
## Vector of images
for imgM in images_list:
    v = imgM.flatten(order="f").reshape(-1,1)

## List of images Names (labels)
img=os.listdir("D:/Projects//train")
img_label=[]
for i in range(len(img)):
    img_label.append(img[i].split('.'))
## Genarete list of index and pix of images
id_train=[]
exp_train=[]
for i in range(len(img_label)):
    id_train.append(img_label[i][0])
    exp_train.append(img_label[i][1])

print('identity :',unique(id_train))
print('expretion :',unique(exp_train))
print('pixel of images :',len(v))

identity : ['subject01', 'subject02', 'subject03', 'subject04', 'subject05', 'subject06', 'subject07', 'subject08', 'subject09', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14']
expretion : ['centerlight', 'happy', 'leftlight', 'noglasses', 'normal', 'rightlight', 'sad', 'sleepy', 'surprised', 'wink']
pixel of images : 77760


In [72]:
i=len(v)
e=(len(unique(exp_train))) ## expresion size
p=len(unique(id_train)) ## number of persons
A=np.zeros((i,e,p))
counter = 0
for r in range(p):
    for q in range(e):
        A[:,q,r]=np.squeeze(images_list[counter].flatten(order="f").reshape(-1,1))
        counter +=1

In [73]:
A.shape

(77760, 10, 14)

In [74]:
## HOSVD for tensor A
S, F, G, H = HOSVD(A)


Read Test images and apply face recognition for test images Z 

In [75]:
images_list_test=[]
for imgT in glob.glob("D:/Projects/test/*.gif"):
  imagesT=mpimg.imread(imgT)
  images_list_test.append(imagesT)
## Vector of images
for imgMT in images_list: 
  vT=imgMT.flatten(order="f").reshape(-1,1)

## List of images Names (labels)
imgT=os.listdir("D:/Projects/test")
test_label=[]
for i in range(len(imgT)):
      test_label.append(imgT[i].split('.'))
## Genarete list of index and pix of images
id_test=[]
exp_test=[]
for i in range(len(test_label)):
    id_test.append(test_label[i][0])
    exp_test.append(test_label[i][1])

print('identity test :',unique(id_test))
print('expretion test :',unique(exp_test))
print('pixel of images test :',len(vT))

identity test : ['subject01', 'subject02', 'subject03', 'subject04', 'subject05', 'subject06', 'subject07', 'subject08', 'subject09', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14', 'subject15']
expretion test : ['exist', '1', '10', '11', '2', '3', '4', '5', '6', '7', '8', '9']
pixel of images test : 77760


In [87]:
# Define function for Test images algorithm
def TestImage(z,B,id): 
    predict = []
    norms=[]
    tol = 0.5
    ## Step 1
    Qe = []
    Re = []
    for e in range(B.shape[1]):
        Q, R = np.linalg.qr(B[:,e,:], mode='reduced')
        Qe.append(Q)
        Re.append(R)  
    ## Step 2
    Z_hat = np.dot(F.T, z) 
    ## Step 3
    for e in range(B.shape[1]):
        alfa = np.linalg.inv(Re[e]) @ (Qe[e]).T @ Z_hat
        for p in range(B.shape[2]):
            norm = np.linalg.norm(alfa[e]-H[:,p])
            norms.append(norm)
    ## Classify person 
    for k in range(len(norms)):
        if norms[k] < tol:
            predict.append(id[k])
    return predict

In [88]:
# Calculate for B in algolithm TestImage
B = multiplication(S,G,2)
pred=[]
valid=0
for k in range(len(images_list_test)): 
    z = images_list_test[k].flatten(order="f").reshape(-1,1)
    id=id_train
    pred.append(TestImage(z,B,id))
    
## Calculate Accuracy of Algorithm
    for i in range(len(pred[k])):
        if pred[k][i]==id_test[i]:
            valid= valid + 1 

## Print Results
print('Accuracy is:',(valid/len(id_test))*100)
print('Number persons predicted correct:', valid)
print('Number persons predicted false:', len(images_list_test)- valid)

Accuracy is: 96.0
Number persons predicted correct: 24
Number persons predicted false: 1
